In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ROOT
import numpy as np
from ROOT import TGraphErrors
from ROOT import TVectorT
import matplotlib.pyplot as plt
from matplotlib import ticker
from root_np_functions import *
from plotting_functions import *

Welcome to JupyROOT 6.18/00


In [3]:
from scipy.optimize import curve_fit
def gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def double_gauss(x,a,mu0,sigma0,b,mu1,sigma1):
    return (a * np.exp(-(x - mu0) ** 2 / (2 * sigma0 ** 2)) +
            b * np.exp(-(x - mu1) ** 2 / (2 * sigma1 ** 2)) )

In [5]:
#filename = "../../processing/histograms_reco_NoCuts_output_mom_res_3T_combinedsigma_eta_5_p_6_.root"
filename = "histograms_reco_NoCuts_output_mom_res_sigma_eta_5_p_6_B_3.0.root"
filename_14T = "histograms_reco_NoCuts_output_mom_res_sigma_eta_5_p_6_B_1.4.root"
file = ROOT.TFile.Open(filename)
#file.ls()


eta_binning = {}
eta_binning["edges"],N_eta= TVT_to_numpy(file,"TVT_eta_bin")
eta_binning["centers"], eta_binning["widths"] = get_binning_from_edges(eta_binning["edges"])

mom_binning = {}
mom_binning["edges"],N_mom = TVT_to_numpy(file,"TVT_mom_bin")
mom_binning["centers"], mom_binning["widths"] = get_binning_from_edges(mom_binning["edges"])

N_mom = len(mom_binning["centers"])
N_eta = len(eta_binning["centers"])

eta_colors = get_colors(plt.cm.winter, N_eta)
mom_colors = get_colors(plt.cm.autumn, N_mom)


B_Fields = [1.4,3.0]
B_Strings = ["1p4","3"]
N_divs = N_mom
xlims=[[-0.04,0.04]]
prefixes = ["h1_dph"] #Types of Resolutions
TDirectories = ["dph_histos"]

h1dict_full = {}
for B_String,B in zip(B_Strings,B_Fields):
        if (B==1.4): 
            file = ROOT.TFile.Open(filename_14T)
        if (B==3.0):
            file = ROOT.TFile.Open(filename)
        for s,d in zip(prefixes,TDirectories):
            h1dict_full[s+"edges"],h1dict_full[s+"centers"],h1dict_full[s+"widths"] = get_th1_binning_np(file.Get("%s/%s_et_0_p_0_bin"%(s,d)))

            for eta in range(N_eta):
                for p in range(N_mom):
                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    h1dict_full[identifier+"_vals"],h1dict_full[identifier+"_errors"] = TH1_to_numpy_wErrors(file.Get("%s/%s_et_%i_p_%i_bin"%(s,eta,p),False,True))
                    h1=file.Get("%s/"%(d)+"%s_et_%i_p_%i_bin"%(s,eta,p))
                    h1dict_full[identifier+"_avg"] = h1.GetMean()
                    h1dict_full[identifier+"_sigma"] = h1.GetStdDev()
                    #print(h1.GetMean())
                    
np.save("np_arrays/indv_histos.npy",h1dict_full)

AttributeError: 'TObject' object has no attribute 'GetNbinsX'

In [ ]:
#Set Fit Range for Single Gauss
fit_min_value = -0.005
fit_max_value = 0.005

arr = h1dict_full["h1_dph"+"centers"]
min_array = np.absolute(arr-fit_min_value)
max_array = np.absolute(arr-fit_max_value)
 
min_index = np.argmin(min_array)
max_index = np.argmin(max_array)+1

B_Fields = [1.4,3.0]
B_Strings = ["1p4","3"]
N_divs = N_mom
xlims=[[-0.02,0.02]]

In [ ]:
string_selections = ["reco_NoCuts_"] #Reco Cuts. First two may not sum to 3rd
res_dict = {}
h1dict = h1dict_full

for s,xlim in zip(prefixes,xlims):
    for B_String,B in zip(B_Strings,B_Fields):
        fig = plt.figure(figsize=(18,12))
        plt.xticks([])
        plt.yticks([])
        plt.title(r"$\Delta\varphi$ Distributions (B = %1.1f T)"%(B),fontsize=40,y=1.04,verticalalignment="bottom")
        i=0
        for eta in range(N_eta):

            for p in range(N_mom):
                ax = fig.add_subplot(N_eta,N_divs,i+1)
                i+=1
                
                if eta==0:
                    p_range_string = r"$%1.1f < p < %1.1f$"%(mom_binning["edges"][p],mom_binning["edges"][p+1])
                    plt.text(0.5,1.18,p_range_string,ha="center",va="top",size=20,transform=ax.transAxes)
                if p==0:
                    eta_range_string = r"$%1.1f < \eta < %1.1f$"%(eta_binning["edges"][eta],eta_binning["edges"][eta+1])
                    plt.ylabel(eta_range_string,fontsize=16)
                
                for col in ["b","r","g"]:

                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    plt.errorbar(h1dict[s+"centers"],h1dict[identifier+"_vals"],yerr=h1dict[identifier+"_errors"],fmt=".",color=col,alpha=0.8)
                    ydata = h1dict[identifier+"_vals"]
                    xdata=h1dict[s+"centers"]
                    xdata=xdata[~np.isnan(ydata)]
                    ydata=ydata[~np.isnan(ydata)]
                    popt, pcov = curve_fit(double_gauss, xdata, ydata, p0=[ydata.max(), xdata.mean(), xdata.std()/4,ydata.max()/5,xdata.mean(),xdata.std()])
                    plt.plot(xdata, double_gauss(xdata, *popt), 'r-', label='fit')
                    #popt, pcov = curve_fit(gauss, xdata, ydata, p0=[ydata.max(), xdata.mean(), xdata.std()/4])
                    #plt.plot(xdata, gauss(xdata, *popt), 'r-', label='fit')
                    res_dict[identifier+"_dDeltaPhi_res"] = popt[5] #narrow gauss sigma
                    #aesthetics
                    plt.xlim(xlim)
                    plt.xticks(fontsize=8)
                    plt.yticks(fontsize=8)
                    
                    plt.text(0.98,0.95,"B = %1.1f T"%(B),ha="right",va="top",size=15,alpha=0.7,transform=ax.transAxes)
        plt.tight_layout()
        plt.savefig(s+"_"+B_String+"_dpp_Overlays_Fits.pdf")
        #print(popt[5])